In [14]:
# Install needed packages
! pip install opencv-python
! pip install tensorflow

Found existing installation: opencv-python-headless 4.11.0.86
Uninstalling opencv-python-headless-4.11.0.86:
  Successfully uninstalled opencv-python-headless-4.11.0.86


You can safely remove it manually.


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [17]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import numpy as np
import collections

### Step 1: Load Pretrained Model

In [5]:
# Load Pretrained Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model initially

# Add task-specific layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # Replace 4 with the number of your classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


### Step 2: Prepare Your Dataset

In [6]:
# Set the base directory
base_dir = os.getcwd()  # Get the current working directory

# Construct the path to the dataset
dataset_dir = os.path.join(base_dir, 'dataset')

# Construct the path to the training directory
train_dir = os.path.join(dataset_dir, 'train')

# Construct the path to the validation directory
val_dir = os.path.join(dataset_dir, 'val')

# Data augmentation for training
data_augmentation = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# No augmentation for validation, just rescaling
val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Prepare generators
train_generator = data_augmentation.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 3919 images belonging to 4 classes.
Found 395 images belonging to 4 classes.


### Step 3: Train the Model

In [7]:
# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the new layers initially using train_generator and val_generator
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[early_stopping],
    verbose=1
)

# Fine-tune the entire model
base_model.trainable = True  # Unfreeze the base layers

# Recompile with a lower learning rate
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the entire model
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    callbacks=[early_stopping],
    verbose=1
)

# Save the trained model
model.save('best_model.keras')


c:\Users\mika\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 179s 1s/step - accuracy: 0.6043 - loss: 1.0095 - val_accuracy: 0.8278 - val_loss: 0.4934
Epoch 2/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 74s 598ms/step - accuracy: 0.7973 - loss: 0.5502 - val_accuracy: 0.8405 - val_loss: 0.4606
Epoch 3/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 72s 585ms/step - accuracy: 0.8133 - loss: 0.4873 - val_accuracy: 0.8608 - val_loss: 0.4491
Epoch 4/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 74s 597ms/step - accuracy: 0.8296 - loss: 0.4604 - val_accuracy: 0.8633 - val_loss: 0.4370
Epoch 5/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 75s 607ms/step - accuracy: 0.8255 - loss: 0.4652 - val_accuracy: 0.8608 - val_loss: 0.4483
Epoch 6/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 71s 580ms/step - accuracy: 0.8433 - loss: 0.4266 - val_accuracy: 0.8709 - val_loss: 0.4144
Epoch 7/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 70s 570ms/step - accuracy: 0.8524 - loss: 0.4000 - val_accuracy: 0.8785 - val_loss: 0.4165
Epoch 8/15
123/123 ━━━━━━━━━━━━━━━━━━━━ 90s 729ms/step - accuracy: 0.8501 - loss: 0.4

In [9]:
# Construct the path to the test directory
test_dir = os.path.join(dataset_dir, 'test')

# Data augmentation for test set
test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Prepare test generator
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Found 379 images belonging to 4 classes.
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 666ms/step - accuracy: 0.9101 - loss: 0.4970
Test Loss: 0.408337265253067
Test Accuracy: 0.9261213541030884


### Step 4: Real-Time Webcam Prediction

This needs to be run outside Jupyter notebook environment for real time webcam access and output. Test script is in webcam-predirector.py file.

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import collections

# Load the saved model
print ("Loading the model...")
model = tf.keras.models.load_model('best_model.keras')

# Class labels for predictions
class_labels = ['Headtop', 'Helmet', 'Hoodie', 'No headwear']

# Print the model summary
model.summary()

# Open the webcam
print("Opening the webcam...")
cap = cv2.VideoCapture(0)  # 0 is the default camera

# Define the target size for the images
target_size = (224, 224)

# Initialize a buffer for smoothing predictions
predictions_buffer = collections.deque(maxlen=10)

print("Press 'q' to exit the video feed.")

try:
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture video frame. Exiting...")
            break

        # Preprocess the frame
        resized_frame = cv2.resize(frame, target_size)  # Resize to match model input
        img_array = np.expand_dims(resized_frame, axis=0) / 255.0  # Normalize and add batch dimension

        # Make prediction
        predictions = model.predict(img_array, verbose=0)
        class_index = np.argmax(predictions[0])
        prediction_label = class_labels[class_index]
        confidence = predictions[0][class_index] * 100

        # Add prediction to the buffer
        predictions_buffer.append(class_index)

        # Smooth predictions using majority voting
        smoothed_prediction = max(set(predictions_buffer), key=predictions_buffer.count)
        smoothed_label = class_labels[smoothed_prediction]

        # Display the prediction on the frame
        cv2.putText(frame, f"{smoothed_label} ({confidence:.2f}%)", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (75, 75, 75), 2)

        # Show the frame
        cv2.imshow('Hat Detection', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("\nProgram interrupted by the user. Exiting...")

finally:
    # Release the webcam and close windows
    cap.release()
#    cv2.destroyAllWindows()
    print("Resources released, video window closed.")